In [8]:
import mysql
import pandas as pd
from mysql.connector import Error
import time

In [9]:
print("Jupyter Notebook para gestionar la base de datos UrbanGo")

# --- Configuracion de Conexion a la Base de Datos ---
# Estos son los mismos valores definidos en docker-compose.yml
DB_CONFIG = {
    'host': 'mysql',        # El nombre del servicio de MySQL en docker-compose
    'user': 'root',         
    'password': 'root_password', # Contraseña de root configurada en docker-compose
    'database': 'urbango_db' # Nombre de la base de datos a la que nos conectaremos
}

Jupyter Notebook para gestionar la base de datos UrbanGo


In [10]:
# --- SQL para Creación de Tablas ---
# Este script crea la base de datos y las tablas, incluyendo llaves primarias y foraneas.
sql_create_tables = """
-- Crear la base de datos (si no existe)
CREATE DATABASE IF NOT EXISTS urbango_db;
USE urbango_db;

-- Tabla para los usuarios registrados
CREATE TABLE IF NOT EXISTS Usuarios (
    id_usuario INT PRIMARY KEY AUTO_INCREMENT,
    nombre VARCHAR(100) NOT NULL,
    email VARCHAR(100) UNIQUE NOT NULL,
    telefono VARCHAR(20) UNIQUE,
    fecha_registro DATETIME NOT NULL DEFAULT CURRENT_TIMESTAMP
);

-- Tabla para los vehiculos de la flota
CREATE TABLE IF NOT EXISTS Vehiculos (
    id_vehiculo INT PRIMARY KEY AUTO_INCREMENT,
    tipo ENUM('monopatin', 'bicicleta', 'auto') NOT NULL,
    modelo VARCHAR(50) NOT NULL,
    anio INT,
    estado ENUM('disponible', 'en_uso', 'mantenimiento') NOT NULL DEFAULT 'disponible'
);

-- Tabla para las estaciones de inicio y fin
CREATE TABLE IF NOT EXISTS Estaciones (
    id_estacion INT PRIMARY KEY AUTO_INCREMENT,
    nombre_estacion VARCHAR(100) UNIQUE NOT NULL,
    ciudad VARCHAR(50) NOT NULL,
    latitud DECIMAL(9, 6),
    longitud DECIMAL(9, 6)
);

-- Tabla para el historial de viajes
CREATE TABLE IF NOT EXISTS Viajes (
    id_viaje INT PRIMARY KEY AUTO_INCREMENT,
    id_usuario INT NOT NULL,
    id_vehiculo INT NOT NULL,
    id_estacion_origen INT NOT NULL,
    id_estacion_destino INT NOT NULL,
    fecha_inicio DATETIME NOT NULL,
    fecha_fin DATETIME,
    costo DECIMAL(8, 2),
    metodo_pago VARCHAR(50),
    FOREIGN KEY (id_usuario) REFERENCES Usuarios(id_usuario),
    FOREIGN KEY (id_vehiculo) REFERENCES Vehiculos(id_vehiculo),
    FOREIGN KEY (id_estacion_origen) REFERENCES Estaciones(id_estacion),
    FOREIGN KEY (id_estacion_destino) REFERENCES Estaciones(id_estacion),
    CONSTRAINT chk_estaciones CHECK (id_estacion_origen != id_estacion_destino)
);
"""

In [11]:
# --- SQL para Truncar Tablas (Limpiar datos existentes) ---
sql_truncate_tables = """
USE urbango_db;
-- Desactivar temporalmente la verificacion de FK para truncar
SET FOREIGN_KEY_CHECKS = 0;

-- Truncar tablas en orden inverso de dependencia para evitar errores de FK
TRUNCATE TABLE Viajes;
TRUNCATE TABLE Vehiculos;
TRUNCATE TABLE Usuarios;
TRUNCATE TABLE Estaciones;

-- Reactivar la verificacion de FK
SET FOREIGN_KEY_CHECKS = 1;
"""

In [12]:
# --- SQL para Insercion de Datos ---
# Datos de prueba para poblar las tablas
sql_insert_data = """
USE urbango_db;

-- 1. Insertar 5 usuarios
INSERT INTO Usuarios (nombre, email, telefono) VALUES
('Ana Lopez', 'ana.lopez@email.com', '5551234567'),
('Juan Perez', 'juan.perez@email.com', '5552345678'),
('Maria Rodriguez', 'maria.rodriguez@email.com', '5553456789'),
('Carlos Sanchez', 'carlos.sanchez@email.com', '5554567890'),
('Sofia Gomez', 'sofia.gomez@email.com', '5555678901');

-- 2. Insertar 10 vehículos (minimo 3 tipos)
INSERT INTO Vehiculos (tipo, modelo, anio, estado) VALUES
('monopatin', 'Xiaomi M365', 2021, 'disponible'),
('monopatin', 'Segway Ninebot', 2022, 'en_uso'),
('monopatin', 'Xiaomi M365', 2021, 'mantenimiento'),
('bicicleta', 'UrbanBike 1', 2020, 'disponible'),
('bicicleta', 'UrbanBike 2', 2021, 'disponible'),
('bicicleta', 'UrbanBike 3', 2022, 'disponible'),
('auto', 'UrbanGo Car', 2019, 'disponible'),
('auto', 'UrbanGo Car', 2020, 'en_uso'),
('auto', 'UrbanGo Car', 2021, 'disponible'),
('monopatin', 'Xiaomi M365', 2022, 'disponible');

-- 3. Insertar 5 estaciones (minimo en 2 ciudades)
INSERT INTO Estaciones (nombre_estacion, ciudad, latitud, longitud) VALUES
('Estación Centro', 'Ciudad A', -34.6037, -58.3816),
('Estación Norte', 'Ciudad A', -34.5772, -58.4239),
('Estación Sur', 'Ciudad A', -34.6293, -58.3758),
('Estación Central', 'Ciudad B', -33.4489, -70.6693),
('Estación Oeste', 'Ciudad B', -33.4589, -70.6893);

-- 4. Insertar 10 viajes
INSERT INTO Viajes (id_usuario, id_vehiculo, id_estacion_origen, id_estacion_destino, fecha_inicio, fecha_fin, costo, metodo_pago) VALUES
(1, 1, 1, 2, '2025-08-11 10:00:00', '2025-08-11 10:15:00', 5.50, 'tarjeta'),
(2, 4, 3, 1, '2025-08-11 11:30:00', '2025-08-11 11:55:00', 8.25, 'paypal'),
(3, 7, 4, 5, '2025-08-11 14:00:00', '2025-08-11 14:45:00', 15.00, 'tarjeta'),
(1, 10, 2, 3, '2025-08-11 16:20:00', '2025-08-11 16:35:00', 4.75, 'tarjeta'),
(4, 5, 1, 3, '2025-08-11 17:00:00', '2025-08-11 17:25:00', 7.50, 'paypal'),
(5, 8, 5, 4, '2025-08-11 18:00:00', '2025-08-11 18:50:00', 20.00, 'tarjeta'),
(2, 6, 2, 1, '2025-08-11 19:00:00', '2025-08-11 19:10:00', 3.00, 'tarjeta'),
(3, 9, 3, 2, '2025-08-11 20:00:00', '2025-08-11 20:30:00', 10.00, 'paypal'),
(4, 1, 4, 5, '2025-08-11 21:00:00', '2025-08-11 21:20:00', 6.00, 'tarjeta'),
(5, 4, 5, 4, '2025-08-11 22:00:00', '2025-08-11 22:40:00', 12.00, 'tarjeta');
"""


In [13]:
def execute_sql_script(cursor, sql_script):
    """
    Ejecuta un script SQL que puede contener múltiples sentencias.
    """
    # Eliminar comentarios y dividir por punto y coma, filtrando sentencias vacias
    commands = [cmd.strip() for cmd in sql_script.split(';') if cmd.strip()]
    for command in commands:
        if command:
            try:
                cursor.execute(command)
                print(f"Comando ejecutado con éxito: {command[:70]}...") # Mostrar un fragmento
            except Error as err:
                # Si es un error de "ya existe", lo ignoramos para los CREATE TABLE IF NOT EXISTS
                # O si es un error de "duplicate entry" en una tabla que se espera truncar, lo ignoramos
                if "already exists" in str(err) or "Can't create database" in str(err) or "Duplicate entry" in str(err):
                    print(f"Advertencia: {err}")
                else:
                    print(f"Error al ejecutar: '{command[:70]}...'\nError: {err}")
                    raise # Relanzar otros errores

In [14]:
# --- Conexion y Ejecucion ---
conn = None
cursor = None
max_retries = 10
retry_delay = 5 # seconds

for i in range(max_retries):
    try:
        print(f"Intentando conectar a la base de datos MySQL (intento {i+1}/{max_retries})...")
        conn = mysql.connector.connect(**DB_CONFIG)
        if conn.is_connected():
            cursor = conn.cursor()
            print("Conexion a MySQL exitosa.")

            # --- 1. Ejecutar script de creación de tablas ---
            print("\n--- Creando tablas en la base de datos UrbanGo ---")
            execute_sql_script(cursor, sql_create_tables)
            conn.commit() # Confirmar los cambios de la creación de tablas
            print("Tablas creadas/verificadas correctamente.")

            # --- 2. Limpiar tablas antes de insertar datos ---
            print("\n--- Limpiando datos existentes en las tablas ---")
            execute_sql_script(cursor, sql_truncate_tables)
            conn.commit() # Confirmar la limpieza
            print("Tablas limpiadas correctamente.")

            # --- 3. Ejecutar script de inserción de datos ---
            print("\n--- Insertando datos de prueba ---")
            execute_sql_script(cursor, sql_insert_data)
            conn.commit() # Confirmar los cambios de la inserción de datos
            print("Datos de prueba insertados/verificados correctamente.")

            break # Salir del bucle si la conexión y ejecución son exitosas
    except Error as e:
        print(f"Fallo en la conexion o ejecucion: {e}")
        if i < max_retries - 1:
            print(f"Reintentando en {retry_delay} segundos...")
            time.sleep(retry_delay)
        else:
            print("Numero maximo de reintentos alcanzado. No se pudo conectar a la base de datos.")
            if conn and conn.is_connected():
                conn.close()
            exit() # Salir del script si falla después de varios reintentos
    finally:
        # Asegúrate de cerrar el cursor y la conexión si hubo un error que no relanzó
        if conn and not conn.is_connected(): # Si la conexión se cerró por error
            if cursor:
                cursor.close()
            conn.close()

Intentando conectar a la base de datos MySQL (intento 1/10)...
Conexion a MySQL exitosa.

--- Creando tablas en la base de datos UrbanGo ---
Comando ejecutado con éxito: -- Crear la base de datos (si no existe)
CREATE DATABASE IF NOT EXISTS...
Comando ejecutado con éxito: USE urbango_db...
Comando ejecutado con éxito: -- Tabla para los usuarios registrados
CREATE TABLE IF NOT EXISTS Usua...
Comando ejecutado con éxito: -- Tabla para los vehiculos de la flota
CREATE TABLE IF NOT EXISTS Veh...
Comando ejecutado con éxito: -- Tabla para las estaciones de inicio y fin
CREATE TABLE IF NOT EXIST...
Comando ejecutado con éxito: -- Tabla para el historial de viajes
CREATE TABLE IF NOT EXISTS Viajes...
Tablas creadas/verificadas correctamente.

--- Limpiando datos existentes en las tablas ---
Comando ejecutado con éxito: USE urbango_db...
Comando ejecutado con éxito: -- Desactivar temporalmente la verificacion de FK para truncar
SET FOR...


Comando ejecutado con éxito: -- Truncar tablas en orden inverso de dependencia para evitar errores ...
Comando ejecutado con éxito: TRUNCATE TABLE Vehiculos...
Comando ejecutado con éxito: TRUNCATE TABLE Usuarios...
Comando ejecutado con éxito: TRUNCATE TABLE Estaciones...
Comando ejecutado con éxito: -- Reactivar la verificacion de FK
SET FOREIGN_KEY_CHECKS = 1...
Tablas limpiadas correctamente.

--- Insertando datos de prueba ---
Comando ejecutado con éxito: USE urbango_db...
Comando ejecutado con éxito: -- 1. Insertar 5 usuarios
INSERT INTO Usuarios (nombre, email, telefon...
Comando ejecutado con éxito: -- 2. Insertar 10 vehículos (minimo 3 tipos)
INSERT INTO Vehiculos (ti...
Comando ejecutado con éxito: -- 3. Insertar 5 estaciones (minimo en 2 ciudades)
INSERT INTO Estacio...
Comando ejecutado con éxito: -- 4. Insertar 10 viajes
INSERT INTO Viajes (id_usuario, id_vehiculo, ...
Datos de prueba insertados/verificados correctamente.


In [15]:

if conn and conn.is_connected():
    print("\n--- Realizando consultas SELECT y mostrando DataFrames ---")

    queries = {
        "Usuarios": "SELECT * FROM Usuarios;",
        "Vehiculos": "SELECT * FROM Vehiculos;",
        "Estaciones": "SELECT * FROM Estaciones;",
        "Viajes": "SELECT * FROM Viajes;"
    }

    for table_name, query in queries.items():
        try:
            print(f"\nDataFrame de la tabla: {table_name}")
            df = pd.read_sql_query(query, conn)
            display(df)
        except Error as err:
            print(f"Error al consultar la tabla {table_name}: {err}")

    # --- Consulta adicional para validar relaciones ---
    print("\nDataFrame de Viajes con detalles de Usuario, Vehiculo y Estaciones:")
    query_full_trips = """
    SELECT
        V.id_viaje,
        U.nombre AS nombre_usuario,
        Vh.tipo AS tipo_vehiculo,
        Vh.modelo AS modelo_vehiculo,
        EO.nombre_estacion AS estacion_origen,
        ED.nombre_estacion AS estacion_destino,
        V.fecha_inicio,
        V.fecha_fin,
        V.costo,
        V.metodo_pago
    FROM
        Viajes V
    JOIN
        Usuarios U ON V.id_usuario = U.id_usuario
    JOIN
        Vehiculos Vh ON V.id_vehiculo = Vh.id_vehiculo
    JOIN
        Estaciones EO ON V.id_estacion_origen = EO.id_estacion
    JOIN
        Estaciones ED ON V.id_estacion_destino = ED.id_estacion;
    """
    try:
        df_full_trips = pd.read_sql_query(query_full_trips, conn)
        display(df_full_trips)
    except Error as err:
        print(f"Error al consultar viajes con detalles: {err}")

    # Termine cierro y me voy
    if cursor:
        cursor.close()
    if conn:
        conn.close()
    print("\nConexión a la base de datos cerrada.")



--- Realizando consultas SELECT y mostrando DataFrames ---

DataFrame de la tabla: Usuarios


/tmp/ipykernel_946/2861965033.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


,id_usuario,nombre,email,telefono,fecha_registro
0,1,Ana Lopez,ana.lopez@email.com,5551234567,2025-08-14 22:55:08
1,2,Juan Perez,juan.perez@email.com,5552345678,2025-08-14 22:55:08
2,3,Maria Rodriguez,maria.rodriguez@email.com,5553456789,2025-08-14 22:55:08
3,4,Carlos Sanchez,carlos.sanchez@email.com,5554567890,2025-08-14 22:55:08
4,5,Sofia Gomez,sofia.gomez@email.com,5555678901,2025-08-14 22:55:08



DataFrame de la tabla: Vehiculos


/tmp/ipykernel_946/2861965033.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


...

,id_viaje,id_usuario,id_vehiculo,id_estacion_origen,id_estacion_destino,fecha_inicio,fecha_fin,costo,metodo_pago
0,1,1,1,1,2,2025-08-11 10:00:00,2025-08-11 10:15:00,5.50,tarjeta
1,2,2,4,3,1,2025-08-11 11:30:00,2025-08-11 11:55:00,8.25,paypal
2,3,3,7,4,5,2025-08-11 14:00:00,2025-08-11 14:45:00,15.00,tarjeta
3,4,1,10,2,3,2025-08-11 16:20:00,2025-08-11 16:35:00,4.75,tarjeta
4,5,4,5,1,3,2025-08-11 17:00:00,2025-08-11 17:25:00,7.50,paypal
5,6,5,8,5,4,2025-08-11 18:00:00,2025-08-11 18:50:00,20.00,tarjeta
6,7,2,6,2,1,2025-08-11 19:00:00,2025-08-11 19:10:00,3.00,tarjeta
7,8,3,9,3,2,2025-08-11 20:00:00,2025-08-11 20:30:00,10.00,paypal
8,9,4,1,4,5,2025-08-11 21:00:00,2025-08-11 21:20:00,6.00,tarjeta
9,10,5,4,5,4,2025-08-11 22:00:00,2025-08-11 22:40:00,12.00,tarjeta



DataFrame de Viajes con detalles de Usuario, Vehiculo y Estaciones:


/tmp/ipykernel_946/2861965033.py:45: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_full_trips = pd.read_sql_query(query_full_trips, conn)


,id_viaje,nombre_usuario,tipo_vehiculo,modelo_vehiculo,estacion_origen,estacion_destino,fecha_inicio,fecha_fin,costo,metodo_pago
0,1,Ana Lopez,monopatin,Xiaomi M365,Estación Centro,Estación Norte,2025-08-11 10:00:00,2025-08-11 10:15:00,5.50,tarjeta
1,4,Ana Lopez,monopatin,Xiaomi M365,Estación Norte,Estación Sur,2025-08-11 16:20:00,2025-08-11 16:35:00,4.75,tarjeta
2,2,Juan Perez,bicicleta,UrbanBike 1,Estación Sur,Estación Centro,2025-08-11 11:30:00,2025-08-11 11:55:00,8.25,paypal
3,7,Juan Perez,bicicleta,UrbanBike 3,Estación Norte,Estación Centro,2025-08-11 19:00:00,2025-08-11 19:10:00,3.00,tarjeta
4,3,Maria Rodriguez,auto,UrbanGo Car,Estación Central,Estación Oeste,2025-08-11 14:00:00,2025-08-11 14:45:00,15.00,tarjeta
5,8,Maria Rodriguez,auto,UrbanGo Car,Estación Sur,Estación Norte,2025-08-11 20:00:00,2025-08-11 20:30:00,10.00,paypal
6,5,Carlos Sanchez,bicicleta,UrbanBike 2,Estación Centro,Estación Sur,2025-08-11 17:00:00,2025-08-11 17:25:00,7.50,paypal
7,9,Carlos Sanchez,monopatin,Xiaomi M365,Estación Central,Estación Oeste,2025-08-11 21:00:00,2025-08-11 21:20:00,6.00,tarjeta
8,6,Sofia Gomez,auto,UrbanGo Car,Estación Oeste,Estación Central,2025-08-11 18:00:00,2025-08-11 18:50:00,20.00,tarjeta
9,10,Sofia Gomez,bicicleta,UrbanBike 1,Estación Oeste,Estación Central,2025-08-11 22:00:00,2025-08-11 22:40:00,12.00,tarjeta



Conexión a la base de datos cerrada.
